## Exploremos un poco la data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! curl https://meli-data-challenge.s3.amazonaws.com/train.csv.gz --output train.csv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  659M  100  659M    0     0  68.7M      0  0:00:09  0:00:09 --:--:-- 70.8M


In [0]:
! find . -name '*.csv.gz' -print0 | xargs -0 -n1 gzip -d

In [0]:
! ls

sample_data  train.csv


In [0]:
import pandas as pd

In [0]:
train_data = pd.read_csv('train.csv')

In [0]:
train_data.head()

,title,label_quality,language,category
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES


In [0]:
train_data.describe()

,title,label_quality,language,category
count,20000000,20000000,20000000,20000000
unique,19988405,2,2,1588
top,Flash Nikon Sb700 Ttl,unreliable,portuguese,PANTS
freq,2,18815755,10000000,35973


,title,label_quality,language,category
count,20000000,20000000,20000000,20000000
unique,19988405,2,2,1588
top,Flash Nikon Sb700 Ttl,unreliable,portuguese,PANTS
freq,2,18815755,10000000,35973


A ver cuántos productos en español y cuántos en portugués hay...

In [0]:
train_data['language'].value_counts()

portuguese    10000000
spanish       10000000
Name: language, dtype: int64

Ah bueno eso está uniformemente distribuido. Veamos la calidad de la etiqueta, según MeLi había muchísimas unreliables.

In [0]:
train_data['label_quality'].value_counts()

unreliable    18815755
reliable       1184245
Name: label_quality, dtype: int64

Bueno tenemos 1588 categorías, y aparentemente sólo el 0.6% de los productos de todas las categorías están marcados como que la etiqueta es de confianza. <br>
Quiero ver cuántas labels reliables y cuántas unreliables tiene cada categoría. Para eso voy a agregar unas columnas al dataframe que diga si la label es reliable o no. Capaz hay una forma más fácil para hacer lo que quiero hacer pero bueno.

In [0]:
train_data['reliable_label'] = train_data['label_quality'] == 'reliable'
train_data['unreliable_label'] = train_data['label_quality'] == 'unreliable'
train_data.head()

,title,label_quality,language,category,reliable_label,unreliable_label
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS,False,True
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS,False,True
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES,False,True
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS,False,True
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES,False,True


Hago una lista porque quiero pasar por todas las categorías y contar cuántos reliables y unreliables tienen.

In [0]:
categories = list(train_data['category'].unique())
categories.sort()
categories

['3D_GLASSES',
 '3D_PENS',
 '3D_PRINTERS',
 '3D_PRINTER_FILAMENTS',
 'ABDOMINAL_TONING_BELTS',
 'ABS_SENSORS',
 'AB_ROLLER_WHEELS',
 'ACCORDIONS',
 'ACOUSTIC_GUITARS',
 'ACOUSTIC_PANELS',
 'ACTION_CAMERA_MOUNTS',
 'ACTION_FIGURES',
 'ADHESIVE_TAPES',
 'AEROBIC_CRUNCH_MACHINES',
 'AFTERSHAVES',
 'AGOGOS',
 'AIRBAGS',
 'AIRBAG_MODULES',
 'AIRBRUSHES',
 'AIRGUN_PELLETS',
 'AIRSOFT_GUNS',
 'AIR_COMPRESSORS',
 'AIR_CONDITIONERS',
 'AIR_CONDITIONER_REMOTE_CONTROLS',
 'AIR_FRESHENERS',
 'AIR_MATTRESSES',
 'ALARMS_AND_SENSORS',
 'ALARM_CLOCKS',
 'ALL_IN_ONE',
 'ALTERNATORS',
 'ALTERNATOR_PULLEYS',
 'AM_FM_RADIOS',
 'ANALOG_CAMERAS',
 'ANGLE_CLAMPS',
 'ANIMAL_AND_BALL_HOPPERS',
 'ANIMAL_CLIPPERS',
 'ANTIQUE_CHAIRS',
 'ANTIQUE_TAPE_RECORDERS',
 'ANTIQUE_TOY_CARS',
 'ANTIVIRUS_AND_INTERNET_SECURITY',
 'ANTI_STATIC_PLIERS',
 'ANTI_THEFT_STUDS',
 'APERITIFS',
 'AQUARIUM_FILTERS',
 'AQUARIUM_SKIMMERS',
 'ARCHERY_ARROWS',
 'ARCHERY_BOWS',
 'ARTIFICIAL_FEATHERS',
 'ARTIFICIAL_FLOWERS',
 'ARTIFICIAL_PL

Ahora agrupo por categoría.

In [0]:
grouped_categories = train_data.groupby('category')

In [0]:
grouped_categories.head()

,title,label_quality,language,category,reliable_label,unreliable_label
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS,False,True
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS,False,True
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES,False,True
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS,False,True
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES,False,True
...,...,...,...,...,...,...
905723,"(pipoqueira Eletrica Profissional ,maquina De ...",unreliable,portuguese,COMMERCIAL_POPCORN_MACHINES,False,True
924951,Pipoqueira Comercial Profissional Elétrica Bra...,unreliable,portuguese,COMMERCIAL_POPCORN_MACHINES,False,True
976206,Máquina De Pipoca Vending Machime Mais Pipoca ...,unreliable,portuguese,COMMERCIAL_POPCORN_MACHINES,False,True
987813,Hamburguesera Santini 13 Cm . Industrial . Alu...,unreliable,spanish,HAMBURGER_FORMERS,False,True


Se me ocurrió armar un diccionario con las categorías como keys y una lista del tipo '[reliable_counts, unreliable_counts]' como values porque no sé cómo crear un dataframe con esta info sino (quise usar el .value_counts() pero no sabía cómo identificar bien cuál era el count para reliable y cuál para unreliable en la serie resultante, como que no estaba ordenado y además cómo detectaba que no hay etiquetas reliables, por ej).

In [0]:
my_dict = {}
for category in categories:
    my_dict[category] = [grouped_categories.get_group(category)['reliable_label'].sum(),
                         grouped_categories.get_group(category)['unreliable_label'].sum()]
my_dict    

{'3D_GLASSES': [7, 7082],
 '3D_PENS': [0, 507],
 '3D_PRINTERS': [850, 5891],
 '3D_PRINTER_FILAMENTS': [17, 4747],
 'ABDOMINAL_TONING_BELTS': [1, 2308],
 'ABS_SENSORS': [734, 12445],
 'AB_ROLLER_WHEELS': [55, 4695],
 'ACCORDIONS': [2477, 11204],
 'ACOUSTIC_GUITARS': [689, 17743],
 'ACOUSTIC_PANELS': [86, 6753],
 'ACTION_CAMERA_MOUNTS': [0, 7429],
 'ACTION_FIGURES': [8512, 19812],
 'ADHESIVE_TAPES': [1407, 26522],
 'AEROBIC_CRUNCH_MACHINES': [228, 3085],
 'AFTERSHAVES': [15, 6255],
 'AGOGOS': [1, 732],
 'AIRBAGS': [211, 9485],
 'AIRBAG_MODULES': [13, 15631],
 'AIRBRUSHES': [145, 10414],
 'AIRGUN_PELLETS': [30, 13808],
 'AIRSOFT_GUNS': [2642, 16533],
 'AIR_COMPRESSORS': [2888, 26371],
 'AIR_CONDITIONERS': [256, 17944],
 'AIR_CONDITIONER_REMOTE_CONTROLS': [52, 19428],
 'AIR_FRESHENERS': [213, 20021],
 'AIR_MATTRESSES': [1080, 10194],
 'ALARMS_AND_SENSORS': [3785, 19476],
 'ALARM_CLOCKS': [7, 19461],
 'ALL_IN_ONE': [2242, 21533],
 'ALTERNATORS': [238, 18767],
 'ALTERNATOR_PULLEYS': [194, 19

Ahora que tengo el diccionario armo un df con el conteo de reliable y unreliable labels para cada categoría.

In [0]:
label_quality_count = pd.DataFrame.from_dict(my_dict, orient='index', columns=['reliable_count', 'unreliable_count'])
label_quality_count.head()

,reliable_count,unreliable_count
3D_GLASSES,7,7082
3D_PENS,0,507
3D_PRINTERS,850,5891
3D_PRINTER_FILAMENTS,17,4747
ABDOMINAL_TONING_BELTS,1,2308


In [0]:
no_reliable_labels = (label_quality_count['reliable_count'] == 0).sum()
no_reliable_labels

193

In [0]:
label_quality_count['reliable_percentage'] = label_quality_count['reliable_count']/(label_quality_count['reliable_count']+label_quality_count['unreliable_count'])
print(label_quality_count['reliable_percentage'].mean())
label_quality_count.sort_values(by='reliable_percentage', ascending=False)

0.041839836222248274


,reliable_count,unreliable_count,reliable_percentage
INDUSTRIAL_BLENDERS,300,269,0.527241
MUSICAL_KEYBOARD_CASES_AND_BAGS,3828,5325,0.418224
FOOTBALL_SHIRTS,8959,18934,0.321192
JACKETS_AND_COATS,8506,18012,0.320763
HANDBAGS,8469,17998,0.319983
...,...,...,...
OXYGEN_TANKS,0,1608,0.000000
FACE_POWDERS,0,9777,0.000000
FALSE_EYELASHES,0,8163,0.000000
COOKING_PLOW_DISCS,0,4404,0.000000


Ok, tengo 193 de las 1588 categorías sin etiquetas confiables, como un 12%, pensé que iba a ser peor. De todas formas en promedio hay sólo un 4% de etiquetas reliables. Habría que ver qué hacer con eso, podríamos hacer nosotros una revisión de los títulos de productos y fijarnos si encajan con la etiqueta asignada o no.
<br>
Puedo usar este df para ver cuántos productos hay en cada categoría también. Ahora el nombre no le queda tan bien, pero bueno.

In [0]:
label_quality_count['total_products'] = label_quality_count['reliable_count'] + label_quality_count['unreliable_count']
label_quality_count.sort_values(by='total_products', ascending=False)

,reliable_count,unreliable_count,reliable_percentage,total_products
PANTS,8020,27953,0.222945,35973
COFFEE_MAKERS,5467,29637,0.155737,35104
BABY_CAR_SEATS,5644,28519,0.165208,34163
MUSICAL_KEYBOARDS,5160,28062,0.155319,33222
MATTRESSES,5552,27415,0.168411,32967
...,...,...,...,...
CONSTRUCTION_LIME_BAGS,0,206,0.000000,206
COLD_FOOD_AND_DRINK_VENDING_MACHINES,0,162,0.000000,162
PAINTBALL_SMOKE_GRENADES,0,154,0.000000,154
COMMERCIAL_POPCORN_MACHINES,0,141,0.000000,141


Comparemos ítems con etiquetas reliables y con etiquetas unreliables dentro de la misma categoría.

In [0]:
grouped_categories.get_group('INDUSTRIAL_BLENDERS').head(15)

,title,label_quality,language,category,reliable_label,unreliable_label
90219,"Liquidificador Industrial 1,5 Litros - Alta Ro...",unreliable,portuguese,INDUSTRIAL_BLENDERS,False,True
111281,Vendo Liquidificador Industrial,unreliable,portuguese,INDUSTRIAL_BLENDERS,False,True
120450,Licuadora Industrial Metvisa 4l Alta Rotacion,unreliable,spanish,INDUSTRIAL_BLENDERS,False,True
124928,Liquidificador Comercial Ls-04mb-n 220v Copo M...,reliable,portuguese,INDUSTRIAL_BLENDERS,True,False
163408,Liquidicador Industrial 2 Lts Inox Com Inmetro...,reliable,portuguese,INDUSTRIAL_BLENDERS,True,False
190323,Licuadora Metvisa Acero Inoxidable Gastronómico,unreliable,spanish,INDUSTRIAL_BLENDERS,False,True
233839,"Liquidificador Industrial Metvisa Lql,04 4 Lit...",reliable,portuguese,INDUSTRIAL_BLENDERS,True,False
318260,"Liquidificador Industrial Skymsen 08mb, 8l, In...",reliable,portuguese,INDUSTRIAL_BLENDERS,True,False
328526,"Lc3 Liquidificador Comercial Inox, Copo Monobl...",reliable,portuguese,INDUSTRIAL_BLENDERS,True,False
373048,"Liquidificador Industrial Alta Rotação 1,5 L S...",reliable,portuguese,INDUSTRIAL_BLENDERS,True,False


A simple vista no parece haber nada raro, todas dicen 'licuadora industrial', no veo por qué no serían confiables. Hay un par que ni dicen 'licuadora' en el título y aún así hay una que es reliable. No sé tengo que pensar mejor cómo se podría analizar esto, o si hay una forma de revisarlos sin que sea a ojo. Capaz se podría chequear que en el nombre estén las palabras 'licuadora', 'industrial' y/o 'comercial' y a esas ponerlas como que son reliables. A ver voy a revisar otra categoría.

In [0]:
grouped_categories.get_group('JACKETS_AND_COATS').head(15)

,title,label_quality,language,category,reliable_label,unreliable_label
945,"Jaqueta Couro Legítimo Masculino, Promoção, Cl...",unreliable,portuguese,JACKETS_AND_COATS,False,True
1908,Campera Bomber Rompevientos Marrón Con Capucha,unreliable,spanish,JACKETS_AND_COATS,False,True
3276,Lindo Piloto Marca Kill Amarillo,unreliable,spanish,JACKETS_AND_COATS,False,True
4864,Campera Ecocuero Diseño Entallado Con Alforzas...,reliable,spanish,JACKETS_AND_COATS,True,False
5269,Jaqueta Feminina Chamuá Camurça Resinada - Sup...,unreliable,portuguese,JACKETS_AND_COATS,False,True
5909,"Tapado De Piel Pars Niña, Con Cartera",unreliable,spanish,JACKETS_AND_COATS,False,True
6562,Campera Rustica Gris Hombre 1 Unidad,unreliable,spanish,JACKETS_AND_COATS,False,True
8250,Campera Mujer Vintage Noventosa Rosa - Envío G...,unreliable,spanish,JACKETS_AND_COATS,False,True
8417,Campera Hombre Entallada,reliable,spanish,JACKETS_AND_COATS,True,False
8767,Jaqueta Jeans Destroyed Lerrux,unreliable,portuguese,JACKETS_AND_COATS,False,True


Este capaz sea un poco más complicado porque es una categoría un poco más amplia que la otra. Sin embargo a mi criterio casi todas esas estarían bien etiquetadas.

Veamos ahora cuántas palabras en promedio hay en los títulos de cada categoría. Creo que voy a hacer algo parecido a lo que hice para las labels.

In [0]:
label_quality_count.loc[['PANTS']]

,reliable_count,unreliable_count,reliable_percentage,total_products
PANTS,8020,27953,0.222945,35973


In [0]:
import re

label_quality_count['avg_word_count'] = ''
for category in categories:
    titles = grouped_categories.get_group(category)['title']
    num_words = titles.apply(lambda x : len(re.findall(r'\w+', x)))
    label_quality_count.loc[[category], 'avg_word_count'] = num_words.mean()
label_quality_count.sort_values(by='avg_word_count', ascending=False)

,reliable_count,unreliable_count,reliable_percentage,total_products,avg_word_count
USB_HUBS,2,15088,0.000133,15090,10.6093
LENS_HOODS,14,7151,0.001954,7165,10.5618
HARD_DRIVE_AND_SSD_ADAPTER_CABLES,41,466,0.080868,507,10.4241
CAMERA_LENSES,2301,18157,0.112474,20458,10.3865
SURVEILLANCE_MONITORING_KITS,1500,11143,0.118643,12643,10.3283
...,...,...,...,...,...
PUREBRED_DOGS,7262,25666,0.220542,32928,5.5475
ELLIPTICAL_MACHINES,1975,7303,0.212869,9278,5.53686
STEPPERS,221,2121,0.094364,2342,5.37062
ROOSTERS_AND_HENS,17,3985,0.004248,4002,5.27536


Probemos armar el dataframe de una manera más agradable, sin tener que armar el diccionario.

In [0]:
reli_counts = train_data.groupby('category')['label_quality'].value_counts()

In [0]:
reli_counts['3D_GLASSES']

label_quality
unreliable    7082
reliable         7
Name: label_quality, dtype: int64

In [0]:
cat_info = pd.DataFrame()
for category in categories:
    cat_info.loc[category, 'unreliable_count'] = reli_counts[category].get('unreliable', default=0)
    cat_info.loc[category, 'reliable_count'] = reli_counts[category].get('reliable', default=0)
cat_info

,unreliable_count,reliable_count
3D_GLASSES,7082.0,7.0
3D_PENS,507.0,0.0
3D_PRINTERS,5891.0,850.0
3D_PRINTER_FILAMENTS,4747.0,17.0
ABDOMINAL_TONING_BELTS,2308.0,1.0
...,...,...
WRISTWATCHES,19082.0,8924.0
XENON_KITS,8352.0,111.0
YARNS,7518.0,708.0
YOGA_MATS,5247.0,36.0


In [0]:
cat_info['total_products'] = cat_info['reliable_count'] + cat_info['unreliable_count']

In [0]:
lang_counts = train_data.groupby('category')['language'].value_counts()
lang_counts

category       language  
3D_GLASSES     portuguese    5116
               spanish       1973
3D_PENS        spanish        330
               portuguese     177
3D_PRINTERS    spanish       3791
                             ... 
YARNS          spanish       3875
YOGA_MATS      spanish       2723
               portuguese    2560
YOGURT_MAKERS  spanish        530
               portuguese     331
Name: language, Length: 3150, dtype: int64

In [0]:
for category in categories:
    cat_info.loc[category, 'spanish'] = lang_counts[category].get('spanish', default=0)
    cat_info.loc[category, 'portuguese'] = lang_counts[category].get('portuguese', default=0)
cat_info

,unreliable_count,reliable_count,total_products,spanish,portuguese
3D_GLASSES,7082.0,7.0,7089.0,1973.0,5116.0
3D_PENS,507.0,0.0,507.0,330.0,177.0
3D_PRINTERS,5891.0,850.0,6741.0,3791.0,2950.0
3D_PRINTER_FILAMENTS,4747.0,17.0,4764.0,4189.0,575.0
ABDOMINAL_TONING_BELTS,2308.0,1.0,2309.0,873.0,1436.0
...,...,...,...,...,...
WRISTWATCHES,19082.0,8924.0,28006.0,17882.0,10124.0
XENON_KITS,8352.0,111.0,8463.0,3637.0,4826.0
YARNS,7518.0,708.0,8226.0,3875.0,4351.0
YOGA_MATS,5247.0,36.0,5283.0,2723.0,2560.0
